In [1]:
import sys
sys.path.insert(0,'C:\\Users\\mamitchell\\git\\simpeg\\discretize')
# # # sys.path.remove('/path/to/mod_directory')
import discretize
print(discretize.__version__)
print(discretize.__path__)

sys.path.insert(1,'C:\\Users\\mamitchell\\git\\simpeg\\simpeg')
import SimPEG
print(SimPEG.__version__)
print(SimPEG.__path__)

print(sys.path)

0.6.2
['C:\\Users\\mamitchell\\git\\simpeg\\discretize\\discretize']
0.14.3
['C:\\Users\\mamitchell\\git\\simpeg\\simpeg\\SimPEG']
['C:\\Users\\mamitchell\\git\\simpeg\\discretize', 'C:\\Users\\mamitchell\\git\\simpeg\\simpeg', 'C:\\Users\\mamitchell\\OneDrive - DOI\\CLVF\\Gravity\\Inv_GroundGrav_Combined', 'C:\\Users\\mamitchell\\Anaconda3\\python38.zip', 'C:\\Users\\mamitchell\\Anaconda3\\DLLs', 'C:\\Users\\mamitchell\\Anaconda3\\lib', 'C:\\Users\\mamitchell\\Anaconda3', '', 'C:\\Users\\mamitchell\\Anaconda3\\lib\\site-packages', 'C:\\Users\\mamitchell\\Anaconda3\\lib\\site-packages\\simpeg-0.14.2-py3.8.egg', 'C:\\Users\\mamitchell\\Anaconda3\\lib\\site-packages\\empymod-2.0.3-py3.8.egg', 'C:\\Users\\mamitchell\\Anaconda3\\lib\\site-packages\\discretize-0.5.1-py3.8-win-amd64.egg', 'C:\\Users\\mamitchell\\Anaconda3\\lib\\site-packages\\vectormath-0.2.2-py3.8.egg', 'C:\\Users\\mamitchell\\Anaconda3\\lib\\site-packages\\properties-0.6.1-py3.8.egg', 'C:\\Users\\mamitchell\\Anaconda3\\lib

In [2]:
from discretize import TreeMesh, utils
from discretize.utils import meshutils
from discretize.utils import active_from_xyz

# from SimPEG import data, data_misfit, directives, maps, inverse_problem, optimization, inversion, regularization
# from SimPEG.potential_fields import gravity

from scipy.interpolate import griddata
import verde
import pandas as pd
import geopandas as gpd
import vtk
import pyvista

import pyproj
import time
import numpy as np

%pylab inline
%matplotlib inline
matplotlib.rcParams['font.size'] = 14


Populating the interactive namespace from numpy and matplotlib


## Load shape file with geopandas

In [3]:
fault_shape = gpd.read_file('../../Datasets/GIS/qfaults_CLVF_10km.shp')
# fault_shape = gpd.read_file('../../Datasets/GIS/qfaults.shp')
fault_shape

C:\Users\mamitchell\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,fault_id,section_id,faultname,sectionnam,dip_direct,mapped_sca,age,sliprate,slipsense,cooperator,linetype,url,Shape_Leng,geometry
0,None,None,None,None,None,"1:100,000",undifferentiated Quaternary,None,None,California Geological Survey,Well Constrained,https://earthquake.usgs.gov/cfusion/qfault/sho...,0.050936,"LINESTRING (-122.95382 38.35647, -122.95697 38..."
1,None,None,None,None,None,"1:100,000",undifferentiated Quaternary,None,None,California Geological Survey,Well Constrained,https://earthquake.usgs.gov/cfusion/qfault/sho...,0.029944,"LINESTRING (-122.99541 38.36599, -122.99040 38..."
2,None,None,None,None,None,"1:24,000",late Quaternary,None,None,California Geological Survey,Well Constrained,https://earthquake.usgs.gov/cfusion/qfault/sho...,0.041239,"LINESTRING (-122.73783 38.77165, -122.72023 38..."
3,None,None,None,None,None,"1:24,000",undifferentiated Quaternary,None,None,California Geological Survey,Inferred,https://earthquake.usgs.gov/cfusion/qfault/sho...,0.001212,"LINESTRING (-122.85427 38.28007, -122.85515 38..."
4,None,None,None,None,None,"1:24,000",undifferentiated Quaternary,None,None,California Geological Survey,Inferred,https://earthquake.usgs.gov/cfusion/qfault/sho...,0.043615,"LINESTRING (-122.65513 38.28629, -122.65881 38..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4202,861,None,unnamed faults near Quackenbush Mountain,None,None,"1:24,000",undifferentiated Quaternary,None,None,California Geological Survey,Well Constrained,https://earthquake.usgs.gov/cfusion/qfault/sho...,0.011017,"LINESTRING (-122.59767 38.95780, -122.59862 38..."
4203,861,None,unnamed faults near Quackenbush Mountain,None,None,"1:24,000",undifferentiated Quaternary,None,None,California Geological Survey,Well Constrained,https://earthquake.usgs.gov/cfusion/qfault/sho...,0.016897,"LINESTRING (-122.59164 38.95771, -122.59218 38..."
4204,861,None,unnamed faults near Quackenbush Mountain,None,None,"1:24,000",undifferentiated Quaternary,None,None,California Geological Survey,Well Constrained,https://earthquake.usgs.gov/cfusion/qfault/sho...,0.012606,"LINESTRING (-122.58699 38.95771, -122.58856 38..."
4205,32,None,Rodgers Creek fault,None,Vertical,Unspecified,undifferentiated Quaternary,Greater than 5.0 mm/yr,Right lateral,California Geological Survey,Well Constrained,https://earthquake.usgs.gov/cfusion/qfault/sho...,0.002845,"LINESTRING (-122.52149 38.24711, -122.52100 38..."


## Setup for coordinate transformation 


In [4]:
fault_shape.crs

C:\Users\mamitchell\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
crs_WGS84 = pyproj.CRS('EPSG:4326')
crs_WGS84

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
crs_UTM_WGS84_10N = pyproj.CRS('EPSG:32610')
crs_UTM_WGS84_10N

<Projected CRS: EPSG:32610>
Name: WGS 84 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - N hemisphere - 126°W to 120°W - by country
- bounds: (-126.0, 0.0, -120.0, 84.0)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
transformer_WGS84_to_WGS84_UTM10N = pyproj.Transformer.from_crs(crs_WGS84, crs_UTM_WGS84_10N)

In [8]:
def makePolyLine(xLoc, yLoc, zLoc):
    # function for creating a vtk line object from points
    nPoints = xLoc.shape[0]
    points = vtk.vtkPoints()
    points.SetNumberOfPoints(nPoints)
    lines = vtk.vtkCellArray()
    lines.InsertNextCell(nPoints)
    for ii in range(0,nPoints):
        points.SetPoint(ii, xLoc[ii],  yLoc[ii],  zLoc[ii])
        lines.InsertCellPoint(ii)

    polyLine = vtk.vtkPolyData()
    polyLine.SetPoints(points)
    polyLine.SetLines(lines)
    
    return polyLine

In [9]:
# Loop through fault_shape data structure, extract geometry of points, transform to UTM, put new locations into vtkPolyLine
xyLoc_Dict = {}
readCount = 0
writeCount = 0
for faultGeom in fault_shape.geometry:
    # fault_id = fault_shape.fault_id[count]
    # print(fault_id)
    faultGeom = fault_shape.geometry[readCount]
    readCount += 1
#     if(readCount > 4):
#         break
        
    if(faultGeom.type == 'MultiLineString'):
        for subfaultGeom in faultGeom:
            subfaultGeom = np.array(subfaultGeom)
            xLoc = subfaultGeom[:,0]
            yLoc = subfaultGeom[:,1]
            zLoc = np.zeros_like(xLoc)
            xLoc_UTM10N, yLoc_UTM10N = transformer_WGS84_to_WGS84_UTM10N.transform(yLoc, xLoc)
#             xyArray = np.vstack([xLoc_UTM10N,yLoc_UTM10N,np.zeros_like(xLoc_UTM10N)]).T
#             vtkDataObj = pyvista.PolyData(xyArray)
            vtkDataObj = makePolyLine(xLoc_UTM10N, yLoc_UTM10N, zLoc)
            xyLoc_Dict[str(writeCount)] = vtkDataObj
            writeCount += 1
    else:        
        faultGeom = np.array(faultGeom)
#         print(faultGeom)
        xLoc = faultGeom[:,0]
        yLoc = faultGeom[:,1]
        zLoc = np.zeros_like(xLoc)
        xLoc_UTM10N, yLoc_UTM10N = transformer_WGS84_to_WGS84_UTM10N.transform(yLoc, xLoc)
#         xyArray = np.vstack([xLoc_UTM10N,yLoc_UTM10N,np.zeros_like(xLoc_UTM10N)]).T
#         vtkDataObj = pyvista.PolyData(xyArray, line=True)
        vtkDataObj = makePolyLine(xLoc_UTM10N, yLoc_UTM10N, zLoc)
        xyLoc_Dict[str(writeCount)] = vtkDataObj
        writeCount += 1




In [10]:
nFaults = len(xyLoc_Dict)
print(nFaults)
# nFaults = 5
vtkFaults = vtk.vtkMultiBlockDataSet()
for ii in range(0,nFaults):
    vtkFaults.SetBlock(ii,xyLoc_Dict[str(ii)])
    

4232


In [11]:
# # writer = vtk.vtkXMLDataObjectWriter()
# # writer.SetFileName('../../Datasets/GIS/qfaults_UTM_Line.vtm')
# # writer.SetInputData(vtkFaults)
# # writer.Write()
faults = pyvista.MultiBlock(vtkFaults)
faults.save('../../Datasets/GIS/qfaults_CLVF_10kmPad_UTM_Line.vtm')

In [12]:
# fault_UTMshape = gpd.read_file('../../Datasets/GIS/qfaults_UTM.shp')
# fault_UTMshape


In [13]:
# count = 0
# fault_UTMGeom = fault_UTMshape.geometry[count]
# fault_UTMGeom = np.array(fault_UTMGeom)
# print(fault_UTMGeom)

In [14]:
# topoXYZ_SRTM_ROI_10kmPad = np.load('./topoXYZ_SRTM_ROI_10kmPad.npy')
# print(topoXYZ_SRTM_ROI_10kmPad.shape)
# xMinTopo = np.floor(np.min(topoXYZ_SRTM_ROI_10kmPad[:,0]))
# xMaxTopo = np.ceil(np.max(topoXYZ_SRTM_ROI_10kmPad[:,0]))
# print('xMinTopo=', xMinTopo)
# print('xMaxTopo=', xMaxTopo)

# yMinTopo = np.floor(np.min(topoXYZ_SRTM_ROI_10kmPad[:,1]))
# yMaxTopo = np.ceil(np.max(topoXYZ_SRTM_ROI_10kmPad[:,1]))
# print('yMinTopo=', yMinTopo)
# print('yMaxTopo=', yMaxTopo)

In [15]:
# latSRTMGrid_ROI_10kmPad = np.load('./latSRTMGrid_ROI_10kmPad.npy')
# lonSRTMGrid_ROI_10kmPad = np.load('./lonSRTMGrid_ROI_10kmPad.npy')
# lonMinTopo = np.floor(np.min(lonSRTMGrid_ROI_10kmPad))
# lonMaxTopo = np.ceil(np.max(lonSRTMGrid_ROI_10kmPad))
# print('lonMinTopo=', lonMinTopo)
# print('lonMaxTopo=', lonMaxTopo)

# latMinTopo = np.floor(np.min(latSRTMGrid_ROI_10kmPad))
# latMaxTopo = np.ceil(np.max(latSRTMGrid_ROI_10kmPad))
# print('latMinTopo=', latMinTopo)
# print('latMaxTopo=', latMaxTopo)